<a href="https://www.kaggle.com/code/dhirajkumarsahu/hindi-tweet-classification?scriptVersionId=102121968" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [95]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindi-tweets-dataset-for-sarcasm-detection/Sarcasm_Hindi_Tweets-SARCASTIC.csv
/kaggle/input/hindi-tweets-dataset-for-sarcasm-detection/scraptweets.py
/kaggle/input/hindi-tweets-dataset-for-sarcasm-detection/README.md
/kaggle/input/hindi-tweets-dataset-for-sarcasm-detection/Scrapping-Hindi-Sarcastic-Tweets.ipynb
/kaggle/input/hindi-tweets-dataset-for-sarcasm-detection/scrap_tweets_in_Hindi-v1.py
/kaggle/input/hindi-tweets-dataset-for-sarcasm-detection/scrap_tweets_in_Hindi-v2.py
/kaggle/input/hindi-tweets-dataset-for-sarcasm-detection/Sarcasm_Hindi_Tweets-NON-SARCASTIC.csv


#### Reference - https://colab.research.google.com/drive/11dFgyNYGn_L1JpclAjBDiEnyZWDLcg41#scrollTo=gqsiXkLFC9hR
#### Dataset - https://www.kaggle.com/datasets/pragyakatyayan/hindi-tweets-dataset-for-sarcasm-detection

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
from tensorflow import keras
from sklearn.metrics import roc_auc_score
import tensorflow
tensorflow.random.set_seed(2)
import re

In [4]:
df_sarcastic = pd.read_csv('../input/hindi-tweets-dataset-for-sarcasm-detection/Sarcasm_Hindi_Tweets-SARCASTIC.csv')
df_non_sarcastic = pd.read_csv('../input/hindi-tweets-dataset-for-sarcasm-detection/Sarcasm_Hindi_Tweets-NON-SARCASTIC.csv')
df_sarcastic['label'] = 'sarcastic'
df_non_sarcastic['label'] = 'non_sarcastic'
df = pd.concat([df_sarcastic, df_non_sarcastic], axis=0)
df = df.drop(['username','acctdesc','location','following','followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts', 'retweetcount', 'hashtags'] ,axis=1)
df = df.reset_index()
df = df.drop('index',axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16179 entries, 0 to 16178
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16179 non-null  object
 1   label   16179 non-null  object
dtypes: object(2)
memory usage: 252.9+ KB


# **Droping Duplicates**

In [5]:
df = df.drop_duplicates(keep='first')

# **Train Test Split**

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], random_state=91)

# **Installing the required Library**

In [8]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 23.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 25.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 49.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 64.8 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.8 MB/s eta 0:00:0000:01:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py)

In [9]:
import ktrain
from ktrain import text
t = text.Transformer("monsoon-nlp/hindi-bert", maxlen=500, class_names=list(set(y_train.values)))

Downloading:   0%|          | 0.00/572 [00:00<?, ?B/s]

# **Text Preprocessing**

In [10]:
trn = t.preprocess_train(X_train.to_numpy(), y_train.to_numpy())

preprocessing train...
language: hi
train sequence lengths:
	mean : 29
	95percentile : 55
	99percentile : 62


/opt/conda/lib/python3.7/site-packages/ktrain/utils.py:738: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  "class_names argument was ignored, as they were extracted from string labels in dataset"


Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/607k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Is Multi-Label? False


In [11]:
evalr = t.preprocess_test(X_test.to_numpy(), y_test.to_numpy())

preprocessing test...
language: hi
test sequence lengths:
	mean : 30
	95percentile : 52
	99percentile : 55


# **Model Training**

In [12]:
model = t.get_classifier()

Downloading:   0%|          | 0.00/59.1M [00:00<?, ?B/s]

In [13]:
learner = ktrain.get_learner(model, train_data=trn, val_data=evalr, batch_size=6)

In [14]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=evalr, batch_size=6)
learner.fit_onecycle(1e-4, 3)



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/3
142/142 [==============================] - 42s 154ms/step - loss: 0.5821 - accuracy: 0.7939 - val_loss: 0.4514 - val_accuracy: 0.8410
Epoch 2/3
142/142 [==============================] - 19s 132ms/step - loss: 0.4292 - accuracy: 0.8327 - val_loss: 0.2460 - val_accuracy: 0.9470
Epoch 3/3
142/142 [==============================] - 19s 132ms/step - loss: 0.1379 - accuracy: 0.9788 - val_loss: 0.0893 - val_accuracy: 0.9859


# **Prediction**

In [51]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

## **Non Sarcastic Examples**

In [90]:
i = 55
print('Actual Text = ', X_test.iloc[i])
print('\n')
print("*"*50)
print("Predicted Label = ",predictor.predict(X_test.iloc[i]))
print('Actual Label = ',y_test.iloc[i])

Actual Text =  कामयाब व्यक्ति अपने चेहरे पर दो ही चीजें रखता है- मुस्कराहट और खामोशी।
#Success
#Smile 
#Silence
#mondaythoughts


**************************************************
Predicted Label =  non_sarcastic
Actual Label =  non_sarcastic


In [91]:
i = 85
print('Actual Text = ', X_test.iloc[i])
print('\n')
print("*"*50)
print("Predicted Label = ",predictor.predict(X_test.iloc[i]))
print('Actual Label = ',y_test.iloc[i])

Actual Text =  #आदि से #अनंत तक बस यही रही है #परंपरा 
कायर भोगे #दुख सदा वीर भोग्य #वसुंधरा

#राजा #रामचंद्र #की #जय 🚩🚩🚩🚩🚩 https://t.co/d3sYXk8hG1


**************************************************
Predicted Label =  non_sarcastic
Actual Label =  non_sarcastic


In [92]:
i = 87
print('Actual Text = ', X_test.iloc[i])
print('\n')
print("*"*50)
print("Predicted Label = ",predictor.predict(X_test.iloc[i]))
print('Actual Label = ',y_test.iloc[i])

Actual Text =  #यादों में तुम••
#ख्वाबों में तुम••
#उदासी मैं तुम••
#खुशी में तुम••
#फिक्र मैं तुम••
#जिक्र मैं तुम••
#बस पास नहीं मेरे तुम•••!!
     (डॉक्टर मुस्तफा अली)
@bhupeshbaghel @ShayarImran @sherryontopp @ShatruganSinha @RTforINDIA @Dr_Uditraj @RajBabbarMP @BBCHindi https://t.co/IsyaNPWub8


**************************************************
Predicted Label =  non_sarcastic
Actual Label =  non_sarcastic


In [93]:
i = 59
print('Actual Text = ', X_test.iloc[i])
print('\n')
print("*"*50)
print("Predicted Label = ",predictor.predict(X_test.iloc[i]))
print('Actual Label = ',y_test.iloc[i])

Actual Text =  Jaane na koi kya chlta kissi ki zindagi me,
Kiis bhoj niche woh daab kr betha hai.
Paar agr us insaan ke yaha hote waqt tumhe koi frk nhi thaa,
Uske jaane ke baad rone ka bhi tumhara haaq nhi..🖕🏻
#Chowkidar_KAMJOR_Hai
#crying #cryingforlove #Sad #ShushantSinghRajput


**************************************************
Predicted Label =  non_sarcastic
Actual Label =  non_sarcastic


## **Sarcastic Examples**

In [83]:

print('Actual Text = ', X_test[520])
print('\n')
print("*"*50)
print("Predicted Label = ",predictor.predict(X_test[520]))
print('Actual Label = ',y_test[520])

Actual Text =  Relatives : " Aur beta padhai kese chal rahi hai .??"
Me : Bass chalti hi jaa rahi hai itni door nikal gayi hai ki mere pohoch se bahat hai....

#sarcastic #relatives #mevsrelative


**************************************************
Predicted Label =  sarcastic
Actual Label =  sarcastic


In [84]:
print('Actual Text = ', X_test[154])
print('\n')
print("*"*50)
print("Predicted Label = ",predictor.predict(X_test[154]))
print('Actual Label = ',y_test[154])

Actual Text =  'आलू से सोना' बनाने की तकनीक खोजने वाले महान वैज्ञानिक की चिंताओं को सिरे से खारिज नहीं किया जा सकता।
@RahulGandhi #RahulTargetsArmy #RahulStillDoubtsIndia #rahultargetarmy #chhotabheem #sarcastic #sarcasm


**************************************************
Predicted Label =  sarcastic
Actual Label =  sarcastic


In [85]:
print('Actual Text = ', X_test[569])
print('\n')
print("*"*50)
print("Predicted Label = ",predictor.predict(X_test[569]))
print('Actual Label = ',y_test[569])

Actual Text =  आज तो चीन की ख़ैर नहीं, टुकड़े-टुकड़े कर देंगे हमारे TV चैनल वाले !
शाम तक कड़ी निंदा भी हो जाएगी, देख लेना !
#Chinese #chinaindiaborder #ChinaIndiaFaceoff #China #justsaying #Sarcasm


**************************************************
Predicted Label =  sarcastic
Actual Label =  sarcastic


In [86]:
print('Actual Text = ', X_test[99])
print('\n')
print("*"*50)
print("Predicted Label = ",predictor.predict(X_test[99]))
print('Actual Label = ',y_test[99])

Actual Text =  @Rashmi_Bajpayee @suryapsinghias Madam ji isko #sarcasm bolte hain aur hum jo kahna chahte the Sir @suryapsinghias samjah gaye 😊 baki smjh smjh ki baat hai koi meri baat ko positively smjhe koi negatively 😀


**************************************************
Predicted Label =  sarcastic
Actual Label =  sarcastic
